In [67]:
import numpy as np
import pandas as pd

In [68]:
class SimilarityMetric:

    def calculateSimilarity(self, u, v):
        pass

In [69]:
class CosineSimilarity(SimilarityMetric):
    def calculateSimilarity(self, u, v):
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v) + 1e-9)

class PCCSimilarity(SimilarityMetric):
    def calculateSimilarity(self, u, v):
        return self.__pearson_correlation(u, v)
    
    def __pearson_correlation(self, user1_ratings, user2_ratings):
        common_items = user1_ratings.index.intersection(user2_ratings.index)
        if len(common_items) < 2:  # Require at least 2 common items for correlation
            return 0
        user1_common_ratings = user1_ratings[common_items]
        user2_common_ratings = user2_ratings[common_items]
        correlation = user1_common_ratings.corr(user2_common_ratings)
        if np.isnan(correlation):
            return 0
        return correlation

In [70]:
class CollaborativeFiltering:

    def __init__(self, data, metric:SimilarityMetric):
        """
        Initialize CollaborativeFiltering object with user-item rating data.
        
        Parameters:
        - data: DataFrame containing user-item ratings
        """
        self.data = data
        self.train_data = None
        self.test_data = None
        self.similarity_matrix = None
        self.metric = metric
    
    def calculate_similarity_matrix(self):
        pass

    def predict_ratings(self, user_id):
        pass

    def train_test_split(self, test_size = 0.2):
        """
        Split the data into training and test sets.
        
        Parameters:
        - test_size: Fraction of the data to be used for testing
        """
        np.random.seed(42)  # for reproducibility
        mask = np.random.rand(len(self.data)) < 1 - test_size
        self.train_data = self.data[mask]
        self.test_data = self.data[~mask]

    def evaluate(self):
        """
        Evaluate the Collaborative Filtering model on the test set.

        Returns:
        - Mean squared error (MSE) of the predictions
        """

        self.calculate_similarity_matrix()
        mse_sum = 0
        total_predictions = 0

        for user_id in self.test_data.index:
            user_test_ratings = self.test_data.loc[user_id]
            user_predicted_ratings = self.predict_ratings(user_id)
            for movie_id, actual_rating in user_test_ratings.items():
                predicted_rating = user_predicted_ratings.loc[movie_id, 'PredictedRating']
                if not pd.isnull(predicted_rating):
                    total_predictions += 1
                    mse_sum += (actual_rating - predicted_rating) ** 2

        mse = mse_sum / total_predictions
        return mse

In [71]:
class CollaborativeFilteringItemItem(CollaborativeFiltering):

    def __init__(self, data, metric:SimilarityMetric):
        super().__init__(data, metric)
    

    def calculate_similarity_matrix(self):
        
        # user_item_matrix = self.train_data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
        
        n_movies = self.data.shape[1]
        similarity_matrix = np.zeros((n_movies, n_movies))
        for i in range(n_movies):
            for j in range(n_movies):
                similarity_matrix[i, j] = self.metric.calculateSimilarity(self.train_data.iloc[:, i], self.train_data.iloc[:, j])
        self.similarity_matrix = pd.DataFrame(similarity_matrix, index=self.train_data.columns, columns=self.train_data.columns)

    def predict_ratings(self, user_id):
        """
        Predict ratings for items for a given user.

        Parameters:
        - user_id: ID of the user for whom to predict ratings

        Returns:
        - DataFrame containing predicted ratings for each item
        """

        user_ratings = self.data.loc[user_id]
        predicted_ratings = pd.DataFrame(index=self.similarity_matrix.columns, columns=['PredictedRating'])

        for movie_id in predicted_ratings.index:
            numerator = 0
            denominator = 0
            for other_movie_id, similarity in self.similarity_matrix[movie_id].items():
                other_movie_rating = user_ratings[other_movie_id]
                if not pd.isnull(other_movie_rating):
                    numerator += similarity * other_movie_rating
                    denominator += abs(similarity)

            
            predicted_ratings.loc[movie_id, 'PredictedRating'] = numerator / (denominator + 1e-9)
            

        return predicted_ratings
        

In [72]:
class CollaborativeFilteringUserUser(CollaborativeFiltering):
    def __init__(self, data, metric:SimilarityMetric):
        
        super().__init__(data, metric)

    def calculate_similarity_matrix(self):
        
        # user_item_matrix = self.train_data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
        n_users = self.data.shape[0]
        similarity_matrix = np.zeros((n_users, n_users))
        for i in range(n_users):
            for j in range(n_users):
                similarity_matrix[i, j] = self.metric.calculateSimilarity(self.train_data.iloc[i, :], self.train_data.iloc[j, :])
                
        self.similarity_matrix = pd.DataFrame(similarity_matrix, index=self.train_data.index, columns=self.train_data.index)

    def predict_ratings(self, user_id):
        """
        Predict ratings for items for a given user.

        Parameters:
        - user_id: ID of the user for whom to predict ratings

        Returns:
        - DataFrame containing predicted ratings for each item
        """

        predicted_ratings = pd.DataFrame(index=self.similarity_matrix.columns, columns=['PredictedRating'])

        for movie_id in predicted_ratings.index:
            numerator = 0
            denominator = 0
            for other_user_id, similarity in self.similarity_matrix[user_id].items():
                other_user_rating = self.data.loc[other_user_id, movie_id]
                if not pd.isnull(other_user_rating):
                    numerator += similarity * other_user_rating
                    denominator += abs(similarity)

            predicted_ratings.loc[movie_id, 'PredictedRating'] = numerator / (denominator + 1e-9)
            

        return predicted_ratings

    


In [73]:
data=pd.read_csv("EncodedCombined.csv")

user_item_matrix = data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
user_item_matrix_new = user_item_matrix.iloc[0:2000, 0:1500]

/tmp/ipykernel_14247/645754126.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("EncodedCombined.csv")


In [74]:
metric = CosineSimilarity()
# Create CollaborativeFiltering instance
cf = CollaborativeFilteringItemItem(user_item_matrix_new, metric)

# Split data into train and test sets
cf.train_test_split(test_size=0.2)

# Evaluate the model
mse = cf.evaluate()
print("Mean Squared Error:", mse)

Mean Squared Error: 0.674702714551718


In [75]:
metric = PCCSimilarity()
cf = CollaborativeFilteringItemItem(user_item_matrix_new, metric)

# Split data into train and test sets
cf.train_test_split(test_size=0.2)

# Evaluate the model
mse = cf.evaluate()
print("Mean Squared Error:", mse)

In [ ]:
metric = CosineSimilarity()
# Create CollaborativeFiltering instance
cf = CollaborativeFilteringUserUser(user_item_matrix_new, metric)


# Split data into train and test sets
cf.train_test_split(test_size=0.2)

# Evaluate the model
mse = cf.evaluate()
print("Mean Squared Error:", mse)

In [ ]:
metric = PCCSimilarity()
cf = CollaborativeFilteringUserUser(user_item_matrix_new, metric)

# Split data into train and test sets
cf.train_test_split(test_size=0.2)

# Evaluate the model
mse = cf.evaluate()
print("Mean Squared Error:", mse)